# Markov Chains

---
In class today we will be implementing a Markov chain to process sentences

---
## Learning Objectives

1. Students will be able to explain the Markov Chain process
1. Implement a Markov Chain


Markov Chains represent a series of events following the Markov Property: future states are memory-less in that they depend only on the current state. This can be expanded to the idea of variable order Markov models where there is a variable-length memory (eg. 1st order Markov Model). Markov models consist of fully observable states. 

> A common example of this is in predicting the weather: We can clearly see the current weather and would like to predict tomorrow's weather. This is also applicable to biology with one case being CpG islands. 

Our goal today will be to implement a Markov model built from words. For our example text, we will use the classic example of Dr. Seuss because of the repetitive nature of the text.

---
## Train Markov model

For our initial implementation of the Markov Model, we will use the simple example of Dr. Seuss: "One fish two fish red fish blue fish."



In [ ]:
def add_entry(markov_model: dict, word_one: str, word_two: str):
    """
    Function to access the markov model and add 1 to word 1's inner dictionary entry for word 2
    This adds 1 to the count of times where word 1 comes before word 2 in the text
    
    @param markov_model: dictionary of dictionaries (may be empty) mapping strings to a dict of {string: integer}
    @param word_one: str, it's the word being used as the key to the outer dictionary
    @param word_two: str, the word being used as the key to the inner dictionary
    """

    # access the outer dictionary's value for word 1 (set it to a blank dictionary if the key doesn't exist)
    markov_model[word_one] = markov_model.get(word_one, {})

    # access the inner dictionary and attempt to access the key for word two (and set the count to 0)
    # regardless of if the value existed before, we're adding 1 to it
    markov_model[word_one][word_two] = markov_model[word_one].get(word_two, 0) + 1 # I SWEAR this isn't as ratchet as it looks!!!!

    # now return the updated model
    return markov_model


def build_markov_model(markov_model: dict, new_text: str):
    '''
    Function to build or add to a 1st order Markov model given a string of text
    We will store the markov model as a dictionary of dictionaries
    The key in the outer dictionary represents the current state
    and the inner dictionary represents the next state with their contents containing
    the transition probabilities.
    Note: This would be easier to read if we were to build a class representation
           of the model rather than a dictionary of dictionaries, but for simplicitiy
           our implementation will just use this structure.
    
    Args: 
        markov_model (dict of dicts): a dictionary of word:(next_word:frequency pairs)
        new_text (str): a string to build or add to the moarkov_model

    Returns:
        markov_model (dict of dicts): an updated markov_model
        
    Pseudocode:
        Add artificial states for start and end
        For each word in text:
            Increment markov_model[word][next_word]
        
    '''

    # split the line into separate words
    sentence = new_text.split()

    # adds the entry for the start state transitioning to the first word in the sentence
    add_entry(markov_model, word_one = "*S*", word_two = sentence[0])

    # iterate over the indices of the words in the sentence
    for i in range(len(sentence) - 1):
        # add the entry for word i transitioning to word i+1
        add_entry(markov_model, word_one = sentence[i], word_two = sentence[i+1])
    
    # now that we're done with the words in the sentence, we just need to add the end state
    add_entry(markov_model, word_one = sentence[-1], word_two = "*E*")

    # return the trained model
    return markov_model

In [3]:
markov_model = dict()
text = "one fish two fish red fish blue fish"
markov_model = build_markov_model(markov_model, text)
print (markov_model)

{'*S*': {'one': 1}, 'one': {'fish': 1}, 'fish': {'two': 1, 'red': 1, 'blue': 1, '*E*': 1}, 'two': {'fish': 1}, 'red': {'fish': 1}, 'blue': {'fish': 1}}


###  Nth order Markov chain
In the above model, each event or word is output from only the previous state with no memory of any prior states. While this is useful in some cases, typical biological applications of Markov chains require higher-order models to accurately capture what we know about a system. For instance, in attempting to identify coding regions of a genome, we know that open reading frames (ORFs) contain codon triplets, and so a third or sixth order Markov chain would better describe these regions. Here you will implement a generalized form of our previous Markov Chain to allow for Nth order chains.


In [1]:
def add_multi(markov_model: dict, key_tuple: tuple[str], next_word: str):
    """
    Function to access the markov model and add 1 to word 1's inner dictionary entry for word 2
    This adds 1 to the count of times where word 1 comes before word 2 in the text
    
    @param markov_model: dictionary of dictionaries (may be empty) mapping strings to a dict of {string: integer}
    @param key_tuple: tuple of strings (variable length), it's the sequences of words being used as the key to the outer dictionary
    @param next_word: str, the word being used as the key to the inner dictionary
    """

    # access the outer dictionary's value for word 1 (set it to a blank dictionary if the key doesn't exist)
    markov_model[key_tuple] = markov_model.get(key_tuple, {})

    # access the inner dictionary and attempt to access the key for word two (and set the count to 0)
    # regardless of if the value existed before, we're adding 1 to it
    markov_model[key_tuple][next_word] = markov_model[key_tuple].get(next_word, 0) + 1 # I SWEAR this isn't as ratchet as it looks!!!!

    # now return the updated model
    return markov_model


def build_markov_model(markov_model, text, order=1):
    '''
    Function to build or add to a Nth order Markov model given a string of text
    Still only takes one line of text as input
    dict mapping tuples of N strings to dictionaries that map strings to ints
    {tuple(str): {str: int}}
    where N is the order

    Args: 
        markov_model (dict of dicts): a dictionary of word:(next_word:frequency pairs)
            or None if a new model is being built
        new_text (str): a string to build or add to the moarkov_model
        order (int): the number of previous states to consider for the model
        
    Returns:
        markov_model (dict of dicts): an updated/new markov_model
    '''

    # split the line of text into a list of words
    sentence = text.split(" ")

    # check if our order is greater than the length of the line
    if order > len(sentence):
        # print the problem to stdout and return the still-untrained model
        print(f"The order {order} is too high for the line length of {len(sentence)} words! Quitting without training the model...")
        return markov_model

    # since we know we're at the start of the line, map the start state to the first word and then slowly flush out the start states
    for i in range(order):
        # we need a decreasing number of start states in our tuple of the outter dictionary

        # make a little list contianing all the instances of the start state that we need for whatever iteration we're currently on
        start_list = ["*S*"] * (order - i)
        # print(f"Start list: {start_list}")

        # iterate over the words in the sentence that we have already hit
        for word in sentence[:i]:
            # add the words we've already hit to the key
            start_list.append(word)
        # print(f"after adding words: {start_list}")

        # now convert the key, currently a list, to a tuple
        key = tuple(start_list)

        # add this key to the markov model
        markov_model[key] = markov_model.get(key, {})

        # add entry to the markov model mapping this key to the next word in the sentence
        add_multi(markov_model, key, sentence[i])

    # now that we're out of that for loop, we can make our keys entirely out of words that are in the list
    # iterate over the indices of the words in the list
    for j in range(len(sentence) - order):
        # prepare the tuple that we'll use as a key, which includes N words from the line
        key = tuple(sentence[j:j+order])
        # print(f"key: {key} // value: {sentence[j+order]}")

        # add an entry mapping the current N words in the window to the next word after
        add_multi(markov_model, key, sentence[j+order])

    # now we are missing the last entry in our dict that maps the last N words to the end state
    key = tuple(sentence[-1 * order:])
    add_multi(markov_model, key, "*E*")

    # return the updated markov model
    return markov_model

    

In [2]:
markov_model = dict()
text = "one fish two fish red fish blue red fish blue"
markov_model = build_markov_model(markov_model, text, order=2)
markov_model

{('*S*', '*S*'): {'one': 1},
 ('*S*', 'one'): {'fish': 1},
 ('one', 'fish'): {'two': 1},
 ('fish', 'two'): {'fish': 1},
 ('two', 'fish'): {'red': 1},
 ('fish', 'red'): {'fish': 1},
 ('red', 'fish'): {'blue': 2},
 ('fish', 'blue'): {'red': 1, '*E*': 1},
 ('blue', 'red'): {'fish': 1}}

## Generate text from Markov Model

Markov models are "generative models". That is, the probability states in the model can be used to generate output following the conditional probabilities in the model.

We will now generate a sequence of text from the Markov model. For this section, I recommend using np.random.choice, which allows for you to provide a probability distribution for drawing the next edge in the chain.

In [35]:
import numpy as np


def calculate_probabilities(word_counts: dict[str, int]):
    """
    Function to calculate the frequency of words, given a dictionary indicating how many times each word occurs

    Frequencies get calculated by summing all the counts in the dict and dividing each count by that sum

    @param word_counts: dictionary mapping strings to integers, the ints are the counts of how many times each of those strings occurs
    @return: dictionary mapping strings to floats, where each float is the frequency (decimal form) of the word
    """

    # calculate the sum of all the words' counts
    total = sum(word_counts.values())  # <- change is that we are overwritiing built in function sum so i changed `sum` to `total`

    # initialize the frequency table
    frequencies = {}

    # now iterate over the key-value pairs in the input dictionary
    for word, count in word_counts.items():  # <- need to add `.items()` as we are attempting to get key and value
        # get the frequency of this word and store it in the dict
        frequencies[word] = count/total # <- again `sum` needs to be `total`

    return frequencies


def get_next_word(current_word, markov_model, seed=42):
    '''
    Function to randomly move a valid next state given a markov model
    and a current state (word)
    
    Args: 
        current_word (tuple): a word that exists in our model
        markov_model (dict of dicts): a dictionary of word:(next_word:frequency pairs)

    Returns:
        next_word (str): a randomly selected next word based on transition probabilies
        
    Pseudocode:
        Calculate transition probilities for all next states from a given state (counts/sum)
            Initialize a probability dictionary
            Access the value in the outer dict associated with our current word
            Once there, we just sum all of the values in current word's inner dictionary
            And then for each word that's a key in the inner dict, we take its count and divide by the sum
            Now in the probability dict, map that key from the inner dict to the number we just didvided for it <3
        Randomly draw from these to generate the next state
            Just pass the values from the probability dict to np.random.choice
        
    '''
    # access the part of the markov model with all the edges that come from the current word
    if current_word not in markov_model:
        return None

    edge_counts = markov_model[current_word]


    # calculate the probabilities for each of the edges
    probability_dict = calculate_probabilities(edge_counts)

    # randomly select words from the probability dict
    next_word = str(np.random.choice(a=list(probability_dict.keys()), p=list(probability_dict.values())))

    # return the word
    return next_word

def generate_random_text(markov_model, seed=42):
    '''
    Function to generate text given a markov model
    
    Args: 
        markov_model (dict of dicts): a dictionary of word:(next_word:frequency pairs)

    Returns:
        sentence (str): a randomly generated sequence given the model
        
    Pseudocode:
        Initialize sentence at start state
        Until End State:
            append get_next_word(current_word, markov_model)
        Return sentence
        
    '''
    # initialize our output string
    generated_text = []

    # start at the start state
    curr_word = "*S*"

    # start a loop that runs until we hit the end state
    while curr_word != "*E*":
        # append the current word
        generated_text.append(curr_word)

        # switch to the next word
        curr_word = get_next_word(curr_word, markov_model, seed)

    # now that we've broken from the loop, we need to append the end state
    generated_text.append(curr_word)
    
    # now join all the text and ship it
    return " ".join(generated_text)

---

## All the Fish
Up till now, you have only been working with a line or two of the Dr. Seuss' _One Fish, Two Fish_. Now, I want you to build a model using the whole book and try different orders of Markov models.

In [40]:

def update_markov_model(markov_model, text, order: int = 1, prev_line: str = None):
    '''
    Function to build or add to a Nth order Markov model given a string of text
    dict mapping tuples of N strings to dictionaries that map strings to ints
    {tuple(str): {str: int}}
    where N is the order

    This assumes that there will be more lines after the one that this reads
    This is resilient against not knowing whether the current line is the first in the file or if it's not

    Args: 
        markov_model (dict of dicts): a dictionary of word:(next_word:frequency pairs)
            or None if a new model is being built
        text (str): a string to build or add to the moarkov_model
        order (int): the number of previous states to consider for the model
        prev_line (str): the previous line of text (optional)
        
    Returns:
        markov_model (dict of dicts): an updated/new markov_model
        tuple of the last N words in the line so the next line isn't treated as the start of the file
    '''

    # split the line of text into a list of words
    sentence = text.split(" ")

    # check if our order is greater than the length of the line
    if order > len(sentence):
        # print the problem to stdout and return the still-untrained model
        print(sentence)
        print(f"The order {order} is too high for the line length of {len(sentence)} words! Quitting without training the model...")
        return markov_model

    # since we know we're at the start of the line, map the start state to the first word and then slowly flush out the start states
    for i in range(order):
        # we need a decreasing number of start states in our tuple of the outter dictionary

        # check if we have a previous line to pull from
        if prev_line:
            # print(f"HEY PREV_LINE WAS TRUE BTW AT I = {i}")

            # take a slice of the last order - i words and make that a list
            start_list = prev_line.split(" ")[-1 * (order - i):]

        else:
            # since we don't have a previous line to pull from, just use the start state (we're at the start of the file)
            # make a little list contianing all the instances of the start state that we need for whatever iteration we're currently on
            start_list = ["*S*"] * (order - i)
            # print(f"Start list: {start_list}")

        # iterate over the words in the sentence that we have already hit
        for word in sentence[:i]:
            # add the words we've already hit to the key
            start_list.append(word)
        # print(f"after adding words: {start_list}")

        # now convert the key, currently a list, to a tuple
        key = tuple(start_list)
        # print(f"Key: {key}")
        

        # add this key to the markov model
        markov_model[key] = markov_model.get(key, {})

        # add entry to the markov model mapping this key to the next word in the sentence
        add_multi(markov_model, key, sentence[i])

    # now that we're out of that for loop, we can make our keys entirely out of words that are in the list
    # iterate over the indices of the words in the list
    for j in range(len(sentence) - order):
        # prepare the tuple that we'll use as a key, which includes N words from the line
        key = tuple(sentence[j:j+order])
        # print(f"key: {key} // value: {sentence[j+order]}")

        # add an entry mapping the current N words in the window to the next word after
        add_multi(markov_model, key, sentence[j+order])

    # now we are missing the last entry in our dict that maps the last N words to the end state
    # key = tuple(sentence[-1 * order:])
    # add_multi(markov_model, key, "*E*")

    # return the updated markov model
    return markov_model


def train_model(model: dict, path: str, order: int = 1):
    """
    Function to train a markov model off of the supplied text file
    Takes a dict as input so you can update a model based on another file without overwriting it

    @param model: dict if empty, or dict mapping tuples to dictionaries of string:int
    @param path: str, the relative path to the text file the model is being trained on
    @param order: int, the order of the Markov model (i.e. how many words long is the key in the outer dict)
    @return: a trained markov model, a dict of dicts, where the key to the outer dict is a tuple of strings 
             and the inner dict maps strings to ints
    """
    # Initialize the previous line
    prev_line = None

    # open the file
    with open(path, mode="r", encoding="utf-8") as book:
        # iterate over the lines of the file
        for line in book:
            # add the data from the 
            trained_model = update_markov_model(model, line, order, prev_line)
            
            # update the previous line before moving to the next
            prev_line=line

    # send the model to the end state
    trained_model = update_markov_model(model, "*E*", order, prev_line=prev_line)

    # pass the trained model
    return trained_model


def generate_text(markov_model, order: int = 1, seed: int = 42):
    '''
    Function to generate text given a markov model
    Why does this exist and how is it different from generate_random_text()?
        *S* and *E* need to be tuples for our parameterized
    
    Args: 
        markov_model (dict of dicts): a dictionary of word:(next_word:frequency pairs)
        order: (int) an int indicating what order of markov model we're working with

    Returns:
        sentence (str): a randomly generated sequence given the model
        
    Pseudocode:
        Initialize sentence at start state
        Until End State:
            append get_next_word(current_word, markov_model)
        Return sentence
        
    '''
    # initialize our output string
    generated_text = []

    # initialize start state
    curr_state = ["*S*"] * order
    curr_word = None

    while True:

        # access whatever the next word is and add it to our generated text
        #print(f"Current state: {curr_state}")
        curr_key = tuple(curr_state)
        #print(f"Current key: {curr_key} // type: {type(curr_key)}")
        curr_word = get_next_word(curr_key, markov_model, seed)
        if curr_word is None or curr_word == "*E*":
            break
        generated_text.append(curr_word)

        # remove the 0th item in the curr_state list and add the next word to the end of the current state
        del curr_state[0]
        curr_state.append(curr_word)
    
    # now join all the text and ship it
    return " ".join(generated_text)



# Now just add some more training data to the markov model. You can find it under data/one_fish_two_fish.txt

markov_model = dict()
# Read in the whole book
text_file = "data/one_fish_two_fish.txt"
first_order = train_model(markov_model, path=text_file, order=3)
#print(first_order)
#for key,val in first_order.items():
#    print(f"Key: {key} // Type: {type(key)} // Value: {val} // Type: {type(val)}")
#print(f"Markov model: {type(first_order)}")
print (generate_text(first_order, order=3, seed=7))

['SO...\n']
The order 3 is too high for the line length of 1 words! Quitting without training the model...
['SO...\n']
The order 3 is too high for the line length of 1 words! Quitting without training the model...
['So...\n']
The order 3 is too high for the line length of 1 words! Quitting without training the model...
['*E*']
The order 3 is too high for the line length of 1 words! Quitting without training the model...
One fish, Two fish, Red fish, Blue fish,
 Black fish, Blue fish, Old fish, New fish.
 This one has a yellow hat.
 From there to here,
 From here to there,
 Funny things are everywhere.
 These yellow pets are called the Zeds.
 They have one hair upon their heads.
 Their hair grows fast. So fast they say,
 They need a haircut every day.
 Who am I? My name is Ish
 On my hand I have a bird I like to hold.
 My shoe is off, my foot is cold.
 My shoe is off, my foot is cold.
 I have a bird I like to do is hop,
 From finger top to finger top.
 I hop from left to right and then.

---
## Pick Your Poison
There are three texts provided for under `data/`. The first is:
1. Dr. Seuss' "One Fist, Two Fish" (179 lines of text)
2. All of Shakespeare's sonnets (2308 lines of text)
3. Homer's "The Odyssey" (9255 Lines of text)

In [45]:
# An example of a more complex text that we can use to generate more complex output
nth_order_markov_model = dict()
#with open("data/odyssey.txt", "r") as poison_text:

    # Process the lines. Consider that sonnets are separated by an empty line.
poison_text = "data/odyssey.txt"
nth_order_markov_model = train_model(markov_model, path = poison_text, order=2)
 
print (generate_text(nth_order_markov_model, order = 2, seed=7))

['you."\n']
The order 2 is too high for the line length of 1 words! Quitting without training the model...
['again."\n']
The order 2 is too high for the line length of 1 words! Quitting without training the model...
['father."\n']
The order 2 is too high for the line length of 1 words! Quitting without training the model...
['me."\n']
The order 2 is too high for the line length of 1 words! Quitting without training the model...
['henceforward."\n']
The order 2 is too high for the line length of 1 words! Quitting without training the model...
['husband.\n']
The order 2 is too high for the line length of 1 words! Quitting without training the model...
['again."\n']
The order 2 is too high for the line length of 1 words! Quitting without training the model...
['it."\n']
The order 2 is too high for the line length of 1 words! Quitting without training the model...
["journey.'\n"]
The order 2 is too high for the line length of 1 words! Quitting without training the model...
["funeral.'\n"]


In [ ]:
# testing